## **Trabajo Práctico Individual n° 2: Aprendizaje Supervisado**
____
__Universidad Tecnológica Nacional, Buenos Aires__\
__Ingeniería Industrial__\
__Ciencia de Datos - Curso I5521__
____

- Nombre: Facundo
- Apellido: Perez
- Legajo: 168410-3

En este trabajo práctico aplicaremos algunas técnicas aprendidas para el aprendizaje automático supervisado.

Utilizaremos un dataset de airbnb donde buscaremos predecir el precio de las noches de cada hogar en funcion de algunas variables.

#### Variables:
- Unnamed: 0: Variable de contador de registros (no será utilizada en nuetro análisis)
- neighbourhood: Barrio donde se encuentra ubicado la casa.
- room_type: Tipo de hogar en alquiler.
- price: Precio por noche.
- minimum_nights: Cantidad minimas de noches que pueden alquilar el hogar.
- number_of_reviews: Cantidad de reviews
- reviews_per_month: Tasa de reviews que tienen por mes
- calculated_host_listings_count: Cantidad de hogares ofrecidos por el host.
- availability_365: Cantidad de dias que se encuentra en alquiler el hogar

## **Objetivos del trabajo practico**
- resolver los requisitos que se detallan en las proximas celdas
- dejar comentarios # en cada linea de codigo que se agregue
- fecha limite de entrega: 13 de Octubre 2022 a las 23h59

In [39]:
## Importo librerías de maniupulación de datos
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [40]:
## Importo librerías de Aprendizaje automático
from sklearn import preprocessing
from sklearn import linear_model
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.utils import shuffle
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR, LinearSVR
from sklearn.metrics import mean_squared_error, make_scorer
from sklearn.model_selection import GridSearchCV

##### **Objetivo 1**
Lo primero que deberan hacer será importar
- el dataset bajo en nombre "london"
- corroborar que lo hayamos cargado correctamente.

In [41]:
#Importo libreria de Google Drive Colab para poder leer el dataset
from google.colab import drive
drive.mount('/content/drive/')

#Importo el dataset
london = pd.read_csv('/content/drive/My Drive/TP2_airbnb_london.csv')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


##### **Objetivo 2**
Ahora imprimir también las dimensiones del dataset y eliminar la variable "Unnamed:0" del dataset.
Volver a guardar el dataset sin esta variable bajo el nombre "london"

In [42]:
london.shape

(51827, 9)

In [43]:
#Como el dataset es muy grande, tomamos una porción del mismo, para realizar más rapido a fines practicos el grid search:

london = london.iloc[:2000,:]


In [44]:
london.head()

,Unnamed: 0,neighbourhood,room_type,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365
0,0,Haringey,Private room,35,1,131,1.71,4,262
1,1,Ealing,Private room,70,2,2,0.07,6,62
2,2,Islington,Private room,45,1,14,0.14,2,364
3,5,Tower Hamlets,Entire home/apt,102,5,7,0.07,56,295
4,6,Barnet,Private room,29,5,114,1.13,3,285


In [45]:
#Usamos la función drop para eliminar la columna/variable unnamed:0
london = london.drop(['Unnamed: 0'], axis=1)

##### **Objetivo 3**
Corroborar que el dataset no cuenta con valores nulos, si los hubiera llenarlos todos con 0.

In [46]:
#Con la función info podemos ver el tipo de dato de cada variable y si contiene nulls
london.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 8 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   neighbourhood                   2000 non-null   object 
 1   room_type                       2000 non-null   object 
 2   price                           2000 non-null   int64  
 3   minimum_nights                  2000 non-null   int64  
 4   number_of_reviews               2000 non-null   int64  
 5   reviews_per_month               2000 non-null   float64
 6   calculated_host_listings_count  2000 non-null   int64  
 7   availability_365                2000 non-null   int64  
dtypes: float64(1), int64(5), object(2)
memory usage: 125.1+ KB


##### **Objetivo 4**
Mostrar el tipo de dato para las variables "neighbourhood" y "room_type".

In [47]:
#con la funcion info ya podemos ver que son tipo object

##### **Objetivo 5**
Generar las variables dummies para estas dos variables categóricas y unirlas al dataset london.


In [48]:
## Genero las dummies para la variable neighbourhood, dentro de la variable neigh_dummies.
# Selecciono primero solo la columna 'Neighbourhood' de la tabla 'london'
neigh_dummies= london[['neighbourhood']].copy()

# Hago get_dummies solo a la columna 'Neighbourhood'
neigh_dummies = pd.get_dummies(data=neigh_dummies, columns=['neighbourhood'])


Imprimir las dimensiones y primeras lineas de estas dummies

In [49]:
neigh_dummies.shape

(2000, 33)

In [50]:
neigh_dummies.head()

,neighbourhood_Barking and Dagenham,neighbourhood_Barnet,neighbourhood_Bexley,neighbourhood_Brent,neighbourhood_Bromley,neighbourhood_Camden,neighbourhood_City of London,neighbourhood_Croydon,neighbourhood_Ealing,neighbourhood_Enfield,...,neighbourhood_Merton,neighbourhood_Newham,neighbourhood_Redbridge,neighbourhood_Richmond upon Thames,neighbourhood_Southwark,neighbourhood_Sutton,neighbourhood_Tower Hamlets,neighbourhood_Waltham Forest,neighbourhood_Wandsworth,neighbourhood_Westminster
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,True,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,True,False,False,False
4,False,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [51]:
## Genero las dummies para la variable room_type, dentro de la variable room_dummies.
# Selecciono primero solo la columna 'room_type' de la tabla 'london'
room_dummies = london[['room_type']].copy()

# Hago get_dummies solo a la columna 'room_type'
room_dummies = pd.get_dummies(data=room_dummies, columns=['room_type'])

Imprimir las dimensiones y primeras lineas de estas dummies

In [52]:
room_dummies.shape

(2000, 3)

In [53]:
room_dummies.head()

,room_type_Entire home/apt,room_type_Private room,room_type_Shared room
0,False,True,False
1,False,True,False
2,False,True,False
3,True,False,False
4,False,True,False


Joinear las variables dummies al dataset



In [54]:

#Lo puedo hacer usando dos veces join por separado, o usando concat:
london = pd.concat([london, room_dummies, neigh_dummies], axis=1)

##### Dividiremos nuestro dataset en la variable dependiente "y" (label o etiqueta) y las variables independientes "x"

In [55]:

y = np.array(london[["price"]])
x = london.drop(['price', 'neighbourhood','room_type'], axis=1)

##### **Objetivo 7**

Dividir el dataset entre Train y Test, tomar un test_size del 30%.

In [56]:
# Train Test Split
xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size=0.30, random_state=42)

##### **Objetivo 8**
Con el StandardScaler generado en la linea siguiente, realizar el transform para el xtrain y xtest guardandolos en nuevas variables llamadas "xtrain_scal" y "xtest_scal".


In [57]:
scaler = preprocessing.StandardScaler().fit(xtrain)

In [58]:
xtrain_scal = scaler.transform(xtrain)

In [59]:
xtest_scal =scaler.transform(xtest)

##### **Objetivo 9**
Definir un modelo de regresion lineal standard y entrenarlo para nuestro dataset (https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html)

Obtener las predicciones y el "mean_squared_error" para el test set.

In [60]:
#Ya importamos LinearRegression al principio
#Generamos la regresion lineal para el conjunto de datos X e Y de entrenamiento
reg = LinearRegression()
reg.fit(xtrain_scal, ytrain)

LinearRegression()

In [61]:
#Visualizamos los parametros de la regresion lineal
reg.coef_


array([[ 1.98278225e+00, -2.30762443e+00, -6.02585937e-01,
         7.70880117e-02,  5.87204362e+00,  1.32360572e+14,
         1.32412740e+14,  1.58189965e+13,  1.46106075e+13,
         4.57541355e+13,  1.19337800e+13,  5.00112695e+13,
         2.52519118e+13,  8.32221588e+13,  1.88487064e+13,
         2.52519118e+13,  5.13440113e+13,  3.46054893e+13,
         4.57541355e+13,  1.01610971e+14,  6.50029917e+13,
         5.69124486e+13,  2.91269359e+13,  8.44147426e+12,
         1.19337800e+13,  3.65580551e+13,  7.82901688e+13,
         7.06963619e+13,  1.88487064e+13,  8.28568873e+13,
         5.80677713e+13,  3.55958787e+13,  4.86378369e+13,
         3.35843912e+13,  3.92957329e+13,  7.20335350e+13,
         8.44147426e+12,  8.46614717e+13,  5.32734840e+13,
         6.29650711e+13,  8.46614717e+13]])

In [62]:
reg.intercept_

array([84.87694697])

In [63]:
#Predicciones de xtest_scal con regresion lineal
reg.predict(xtest_scal)

array([[110.34960322],
       [ 49.62304072],
       [155.81444697],
       [110.19335322],
       [ 30.13085322],
       [141.75194697],
       [126.63085322],
       [ 88.25194697],
       [ 46.38085322],
       [ 52.25585322],
       [111.99804072],
       [ 60.77929072],
       [115.68554072],
       [ 36.19335322],
       [ 54.97460322],
       [141.63085322],
       [143.25194697],
       [ 63.94335322],
       [ 39.44335322],
       [129.69335322],
       [ 61.24804072],
       [ 93.50194697],
       [ 45.63085322],
       [114.27929072],
       [ 54.59960322],
       [123.38085322],
       [129.74804072],
       [ 37.44335322],
       [123.00585322],
       [106.88085322],
       [ 40.94335322],
       [ 64.47460322],
       [165.87694697],
       [ 41.41210322],
       [122.41210322],
       [120.19335322],
       [ 43.97460322],
       [ 37.44335322],
       [ 55.63085322],
       [122.44335322],
       [110.62304072],
       [112.06054072],
       [ 46.56835322],
       [ 57

In [64]:
#Obtenemos el error cuadratico medio
mean_squared_error(ytest, reg.predict(xtest_scal))

1120.2235947437277

##### **Objetivo 10**
Definir un modelo de regresion SVM Regressor (https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVR.html) e implementar GridSearch para realizar la búsqueda de hiperparámetros.

Asignar dos posibles valores para el parámetro "kernel", y tres posibles valores para "C" y "gamma".

Setear al parámetro "refit" en True y hacer 5 CrossValidation.

In [65]:
#Ya importamos SVR Epsilon-Support Vector Regression al principio
#Definimos el modelo
regsvr = SVR()

In [66]:
#Los valores posibles de los parametros son:
#kernel: {‘linear’, ‘poly’, ‘rbf’, ‘sigmoid’, ‘precomputed’} or callable, default=’rbf’
#gamma: {‘scale’, ‘auto’} or float, default=’scale’ -> 1/n_features o float no negativo
#C: float, default=1.0

param_regsvr = {'kernel':('linear', 'rbf'), 'C':[0.1, 90, 1000 ], 'gamma':[ 0.1, 1, 10]}

In [67]:
#https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html
# cv indica la cantidad de folds
# refit nos devuelve el modelo con los mejores parametros encontrados

regressor_svr = GridSearchCV(estimator = regsvr, param_grid = param_regsvr, refit = True, cv = 5,verbose = 1)


In [68]:
regressor_svr.fit(xtrain_scal, ytrain.ravel())

Fitting 5 folds for each of 18 candidates, totalling 90 fits


GridSearchCV(cv=5, estimator=SVR(),
             param_grid={'C': [0.1, 90, 1000], 'gamma': [0.1, 1, 10],
                         'kernel': ('linear', 'rbf')},
             verbose=1)

##### **Objetivo 11**

Mostrar cuales fueron los mejores hiperparámetros y para estos obtener el mean_squared_error en el test set del punto anterior

In [69]:
print("The best parameters are %s with a score of %0.2f" % (regressor_svr.best_params_, regressor_svr.best_score_))

The best parameters are {'C': 90, 'gamma': 0.1, 'kernel': 'linear'} with a score of 0.46


In [70]:
#Obtenemos el error cuadratico medio
mean_squared_error(ytest, regressor_svr.predict(xtest_scal))

1163.8699867623009

##### **Objetivo 12**
Repetir el objetivo 10 pero ahora seteando nuevos valores para los hiperparámetros mencionados con el objetivo de mejorar las predicciones.

Cuales son los mejores hiperparámetros ahora?
Obtenemos una mejora en las predicciones?
Mostrar los resultados.

In [83]:
#Los valores posibles de los parametros son:
#kernel: {‘linear’, ‘poly’, ‘rbf’, ‘sigmoid’, ‘precomputed’} or callable, default=’rbf’
#gamma: {‘scale’, ‘auto’} or float, default=’scale’ -> 1/n_features o float no negativo
#C: float, default=1.0

param_regsvr = {'kernel':('linear', 'poly'), 'C':[100, 90, 150], 'gamma':[ 0.09, 0.1, 0.08]}

In [84]:
#https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html
# cv indica la cantidad de folds
# refit nos devuelve el modelo con los mejores parametros encontrados

regressor_svr = GridSearchCV(estimator = regsvr, param_grid = param_regsvr, refit = True, cv = 5,verbose = 1)

In [85]:
regressor_svr.fit(xtrain_scal, ytrain.ravel())

Fitting 5 folds for each of 18 candidates, totalling 90 fits


GridSearchCV(cv=5, estimator=SVR(),
             param_grid={'C': [100, 90, 150], 'gamma': [0.09, 0.1, 0.08],
                         'kernel': ('linear', 'poly')},
             verbose=1)

In [86]:
print("The best parameters are %s with a score of %0.2f" % (regressor_svr.best_params_, regressor_svr.best_score_))

The best parameters are {'C': 100, 'gamma': 0.09, 'kernel': 'linear'} with a score of 0.46


In [87]:
#Obtenemos el error cuadratico medio
mean_squared_error(ytest, regressor_svr.predict(xtest_scal))

1163.8592946337324

In [82]:
#Obtenemos una leve mejora en los resultados, practicamente imperceptible
#Hay que tener en cuenta que he usado un bajo numero de muestras debido a los tiempos de ejecución.
#Podrian obtenerse mejores resultados incrementando el numero de muestras,
#lo cual tambien nos arrojaría otros valores de hiperparametros.